In [1]:
from glob import glob
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import xgcm
from xorca.lib import load_xorca_dataset
import pickle
import eddytools as et
from cmocean import cm

import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [2]:
# mask
meshpath = ['/scratch/usr/shkifmjr/NUSERDATA/ORION/10-data/meshmask/1_mesh_mask.nc']
mask_10 = xr.open_dataset(meshpath[0], chunks={"y":100, "x":100}).tmask.isel(z=0).squeeze()
mask_10_base = xr.open_dataset('/scratch/usr/shkifmjr/NUSERDATA/ORION/10-data/meshmask/mesh_mask.nc', chunks={"y":100, "x":100}).tmask.isel(z=0).squeeze()

# bathymetry for eddytools
path_bathy = '/scratch/usr/shklvn09/SCRATCH/ORION10.L46.LIM2vp.CFCSF6.MOPS.JRA.XIOS2.5.LP01-EXP05/OUT/'

# observational data
obs_path = '/scratch/usr/shkifmmp/master/data/observations/'

#EKE
path_EXP6 = '/scratch/usr/shkifmmp/manusc/data/EXP06/EKE/'

# DATA DETECTION
datapath= '/scratch/usr/shkifmmp/manusc/data/'

datapath_r = '/scratch/usr/shkifmmp/master/data/'

### TRACKING WITHIN ACC

In [3]:
def f_tracking(type_d, model, year_i, year_e, continuar = False):
    """
    type_d = 0 -> ACC REGION / = 1 -> WHOLE REGION
    model = 0 -> EXP05 / = 1 -> EXP06
    
    """
    if model == 1:
        datapath_t = datapath + 'EXP06/detected/'
    elif model == 0:
        datapath_t = datapath + 'EXP05/detected/' if (type_d == 1) else '/scratch/usr/shkifmmp/master/data/ORION10/eddies_detect/acc/'

    
    if (type_d == 0):
        lat1, lat2= -65, -37
        if model == 1:
            file_root = 'det_typed0'
            file_spec = 'eddies_OW0.3_npixmax421_npixmin10'
        elif model == 0:
            file_root = 'case9'
            file_spec = 'eddies_OW0.3_77.5_69.5'
        
    else:
        lat1, lat2= -67, -30
        file_root = 'det_typed1'
        file_spec = 'eddies_OW0.2_npixmax1038_npixmin10'
        
        
    # Specify parameters for eddy tracking
    tracking_param = {'model': 'ORCA',
                       'grid': 'latlon',
                       'start_time': str(year_i) + '-01-03', # time range start
                       'end_time': str(year_e) + '-12-31', # time range end
                       'calendar': 'standard', # calendar, must be either 360_day or standard
                       'dt': 5, # temporal resolution of the data in days
                       'lon1': 77.5, # minimum longitude of detection region
                       'lon2': 69.5, # maximum longitude
                       'lat1': lat1, # minimum latitude
                       'lat2': lat2, # maximum latitude
                       'dE': 50., # maximum distance of search ellipsis from eddy center in towards the east 
                                 # (if set to 0, it will be calculated as (150. / (7. / dt)))
                       'eddy_scale_min': 0.5, # minimum factor by which eddy amplitude and area are allowed to change in one timestep
                       'eddy_scale_max': 1.5, # maximum factor by which eddy amplitude and area are allowed to change in one timestep
                       'dict': 0, # dictionary containing detected eddies to be used when not stored in files (set to 0 otherwise)
                       'data_path': datapath_t, # path to the detected eddies pickle files
                       'file_root': file_root,
                       'file_spec': file_spec,
                       'ross_path': datapath_r} # path to rossrad.dat containing Chelton et a1. 1998 Rossby radii
    
    if continuar == False:
        tracks_dict, tracks, terminated_list = et.tracking.split_track(tracking_param, in_file=True, continuing=False, tracks=[], terminated_list=[])
        
        
    elif continuar == True:
        
        with open(datapath_t
                  + 'track_19580101_' + str(year_i - 1) + '1231_tracks_typed' + str(type_d)
                  + '.pickle', 'rb') as f:
            tracks = pickle.load(f)
        f.close()
        
        with open(datapath_t
                  + 'terminatedlist_' + str(year_i - 1) + '_tracks_typed' + str(type_d)
                  + '.pickle', 'rb') as f:
            terminated_list = pickle.load(f)
        f.close()
        
        tracks_dict, tracks, terminated_list = et.tracking.split_track(tracking_param, in_file=True, continuing=True, tracks=tracks, terminated_list=terminated_list)

    # We save the tracks for later use
    with open(datapath_t
              + 'track_19580101_' + str(year_e) + '1231_tracks_typed' + str(type_d)
              + '.pickle', 'wb') as f:
        pickle.dump(tracks, f, pickle.HIGHEST_PROTOCOL)
    f.close()
    
    with open(datapath_t
              + 'terminatedlist_' + str(year_e) + '_tracks_typed' + str(type_d)
              + '.pickle', 'wb') as f:
        pickle.dump(terminated_list, f, pickle.HIGHEST_PROTOCOL)
    f.close()
    
    with open(datapath_t
              + 'track_dict_19580101_' + str(year_e) + '1231_tracks_typed' + str(type_d)
              + '.pickle', 'wb') as f:
        pickle.dump(tracks_dict, f, pickle.HIGHEST_PROTOCOL)
    f.close()
    
    return tracks_dict, tracks, terminated_list

In [ ]:
tracks_dict, tracks, terminated_list = f_tracking(type_d=0, model=1, year_i=1981, year_e=1983, continuar=True)

tracking at time step  1  of  219
tracking at time step  25  of  219
